In [8]:

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [9]:

from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

# `pip install llama-index-llms-openai` if you don't already have it
from llama_index.llms.openai import OpenAI


class JokeEvent(Event):
    joke: str


class JokeFlow(Workflow):
    # llm = OpenAI()
    llm = ollama_llm

    @step
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)
        return StopEvent(result=str(response))


w = JokeFlow(timeout=60, verbose=False)
result = await w.run(topic="pirates")
print(str(result))

The joke in question is a play on words, using the nautical term "arrrr" to create a pun. Here's a thorough analysis and critique of the joke:

**Strengths:**

1. **Wordplay**: The use of "arrrr" as a verb (to argue) is a clever play on words. It's a common sound associated with pirates, but in this context, it's used to create a pun.
2. **Simple and easy to understand**: The joke is straightforward and doesn't require complex setup or inside knowledge. This makes it accessible to a wide audience.

**Weaknesses:**

1. **Predictability**: The punchline is fairly predictable, which can make the joke feel less surprising or impactful. A more unexpected twist might have made the joke more memorable.
2. **Lack of originality**: The "arrrr" pun has been used in many other jokes and memes before this one. While it's still a clever play on words, it doesn't bring anything new to the table.
3. **Limited humor scope**: The joke relies heavily on wordplay, which might not appeal to everyone. Some

In [10]:
from llama_index.core.workflow import draw_all_possible_flows

draw_all_possible_flows(w, filename="joke_workflow.html")

joke_workflow.html


C:\Users\Muthu\AppData\Local\Temp\ipykernel_28780\363681527.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) draw_all_possible_flows. (Install `llama-index-utils-workflow` and use the import `from llama_index.utils.workflow` instead.)
  draw_all_possible_flows(w, filename="joke_workflow.html")
